## **orginal dataset- splitting script**

In [4]:
RUN_DATASET_SPLIT = False 

if RUN_DATASET_SPLIT:
    import os
    import random

    BASE_DIR = "  " #  " enter your dataset - folder name that conatins train,val,test folders". here mine is " i.e dataset "

    splits = {
    "train": 0.70,
    "val": 0.15,
    "test": 0.15
    }

# Get class names
    classes = os.listdir(os.path.join(BASE_DIR, "train"))
    classes = [c for c in classes if not c.startswith('.')]

    for cls in classes:
        print(f"Processing class: {cls}")

    # Take image list from train folder
        train_path = os.path.join(BASE_DIR, "train", cls)
        images = [img for img in os.listdir(train_path)
                  if img.lower().endswith(('.jpg', '.png', '.jpeg'))]

        random.shuffle(images)

        total = len(images)
        train_end = int(total * splits["train"])
        val_end = train_end + int(total * splits["val"])

        keep = {
            "train": set(images[:train_end]),
            "val": set(images[train_end:val_end]),
            "test": set(images[val_end:])
        }

    # Clean each split
        for split in ["train", "val", "test"]:
            split_path = os.path.join(BASE_DIR, split, cls)
            for img in os.listdir(split_path):
                if img not in keep[split]:
                    os.remove(os.path.join(split_path, img))

    print("✅ Dataset split completed successfully!")

else:
    print("ℹ️ Dataset split skipped (script runned already, this script was just a reference to see how script's split done.).")


ℹ️ Dataset split skipped (script runned already, this script was just a reference to see how script's split done.).


when we run the script mac will load hidden files i.e, **.DS_store files** which must be removed so i done this below

In [6]:
import os

def remove_ds_store(root_dir):
    removed = 0
    for root, dirs, files in os.walk(root_dir):
        if ".DS_Store" in files:
            ds_path = os.path.join(root, ".DS_Store")
            os.remove(ds_path)
            removed += 1
            print(f"Removed: {ds_path}")
    print(f"\n✅ Total .DS_Store files removed: {removed}")

remove_ds_store("dataset")


Removed: dataset/.DS_Store
Removed: dataset/test/.DS_Store
Removed: dataset/train/.DS_Store
Removed: dataset/val/.DS_Store

✅ Total .DS_Store files removed: 4


In [7]:
import os

base = "dataset"

for split in ["train", "val", "test"]:
    print(f"\n{split.upper()}")
    split_path = os.path.join(base, split)

    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)

        if not os.path.isdir(cls_path):
            continue  # skip files like .DS_Store

        count = len([
            f for f in os.listdir(cls_path)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ])

        print(f"{cls}: {count}")



TRAIN
paper: 597
textile: 689
metal: 630
cardboard: 625
organic: 676
e-waste: 695
glass: 663
plastic: 623

VAL
paper: 127
textile: 147
metal: 135
cardboard: 133
organic: 145
e-waste: 148
glass: 142
plastic: 133

TEST
paper: 129
textile: 149
metal: 136
cardboard: 135
organic: 146
e-waste: 150
glass: 143
plastic: 135


## **binary classification**- recyclable/non-recyclable dataset
This script will:
- ✔ Read from your existing dataset/
- ✔ Create a new binary dataset
- ✔ Keep train/val/test splits intact

**📁 Output structure**

- binary_dataset/
-   ├── train/
-   │   ├── recyclable/
-   │   └── non_recyclable/
-   ├── val/
-   └── test/


In [6]:
RUN_DATASET_SPLIT = False 

if RUN_DATASET_SPLIT:
    import os
    import shutil

    SOURCE_DIR = "dataset"
    TARGET_DIR = "binary_dataset"

    recyclable_classes = [
        "paper", "plastic", "glass",
        "metal", "cardboard", "e-waste"
    ]

    non_recyclable_classes = [
        "organic", "textile"
    ]

    splits = ["train", "val", "test"]

    for split in splits:
        for label in ["recyclable", "non_recyclable"]:
            os.makedirs(os.path.join(TARGET_DIR, split, label), exist_ok=True)

    for split in splits:
        for cls in os.listdir(os.path.join(SOURCE_DIR, split)):
            cls_path = os.path.join(SOURCE_DIR, split, cls)
            if not os.path.isdir(cls_path):
                continue

            if cls in recyclable_classes:
                target_label = "recyclable"
            elif cls in non_recyclable_classes:
                target_label = "non_recyclable"
            else:
                continue

            for img in os.listdir(cls_path):
                shutil.copy(
                    os.path.join(cls_path, img),
                    os.path.join(TARGET_DIR, split, target_label, img)
                )

    print("✅ Binary dataset created successfully!")
else:
    print("ℹ️ Dataset split skipped (script runned already and saved to disk")

ℹ️ Dataset split skipped (script runned already and saved to disk


In [7]:
import os

def remove_ds_store(root_dir):
    removed = 0
    for root, dirs, files in os.walk(root_dir):
        if ".DS_Store" in files:
            ds_path = os.path.join(root, ".DS_Store")
            os.remove(ds_path)
            removed += 1
            print(f"Removed: {ds_path}")
    print(f"\n✅ Total .DS_Store files removed: {removed}")

remove_ds_store("binary_dataset")


Removed: binary_dataset/.DS_Store
Removed: binary_dataset/test/.DS_Store
Removed: binary_dataset/train/.DS_Store
Removed: binary_dataset/val/.DS_Store

✅ Total .DS_Store files removed: 4


In [8]:
import os

base = "binary_dataset"

for split in ["train", "val", "test"]:
    print(f"\n{split.upper()}")
    for cls in os.listdir(os.path.join(base, split)):
        count = len(os.listdir(os.path.join(base, split, cls)))
        print(f"{cls}: {count}")



TRAIN
non_recyclable: 1365
recyclable: 3833

VAL
non_recyclable: 292
recyclable: 818

TEST
non_recyclable: 295
recyclable: 828


## **multi class** - only recyclable dataset
- ✔ Reads from dataset/
- ✔ Copies only recyclable classes
- ✔ Preserves splits
- ✔ Safe to run once

**📁 TARGET STRUCTURE**

- recyclable_dataset/
-    ├── train/
-    │   ├── paper/
-    │   ├── plastic/
-    │   ├── glass/
-    │   ├── metal/
-    │   ├── cardboard/
-    │   └── e-waste/
-    ├── val/
-    └── test/


In [2]:
RUN_DATASET_SPLIT = False 

if RUN_DATASET_SPLIT:
    import os
    import shutil

    SOURCE_DIR = "dataset"
    TARGET_DIR = "recyclable_dataset"

    recyclable_classes = [
        "paper", "plastic", "glass",
        "metal", "cardboard", "e-waste"
    ]

    splits = ["train", "val", "test"]

    # Create folders
    for split in splits:
        for cls in recyclable_classes:
            os.makedirs(
                os.path.join(TARGET_DIR, split, cls),
                exist_ok=True
            )

    # Copy files
    for split in splits:
        split_path = os.path.join(SOURCE_DIR, split)

        for cls in recyclable_classes:
            cls_path = os.path.join(split_path, cls)
            if not os.path.isdir(cls_path):
                continue

            for img in os.listdir(cls_path):
                shutil.copy(
                    os.path.join(cls_path, img),
                    os.path.join(TARGET_DIR, split, cls, img)
                )

    print("✅ Recyclable dataset created successfully!")
else:
    print("ℹ️ Dataset split skipped (script runned already and saved to disk")

ℹ️ Dataset split skipped (script runned already and saved to disk


In [4]:
import os

def remove_ds_store(root_dir):
    removed = 0
    for root, dirs, files in os.walk(root_dir):
        if ".DS_Store" in files:
            ds_path = os.path.join(root, ".DS_Store")
            os.remove(ds_path)
            removed += 1
            print(f"Removed: {ds_path}")
    print(f"\n✅ Total .DS_Store files removed: {removed}")

remove_ds_store("recyclable_dataset")


Removed: recyclable_dataset/.DS_Store
Removed: recyclable_dataset/val/.DS_Store

✅ Total .DS_Store files removed: 2


In [5]:
import os

base = "recyclable_dataset"

for split in ["train", "val", "test"]:
    print(f"\n{split.upper()}")
    for cls in os.listdir(os.path.join(base, split)):
        count = len(os.listdir(os.path.join(base, split, cls)))
        print(f"{cls}: {count}")



TRAIN
paper: 597
metal: 630
cardboard: 625
e-waste: 695
glass: 663
plastic: 623

VAL
paper: 127
metal: 135
cardboard: 133
e-waste: 148
glass: 142
plastic: 133

TEST
paper: 129
metal: 136
cardboard: 135
e-waste: 150
glass: 143
plastic: 135


***all splits were preserved**

## **summary** :
The dataset was prepared to support a hierarchical waste classification pipeline consisting of a binary and a multi-class model.
The original waste image dataset was first split into training (70%), validation (15%), and test (15%) sets in a class-wise manner to avoid data leakage.

To train the binary classifier, waste categories were grouped into:
- Recyclable: cardboard, paper, plastic, glass, metal, e-waste
- Non-recyclable: organic, textile
- Images were reorganized into a binary dataset while keeping the same train/validation/test split.

To train the multi-class classifier, only recyclable waste images were used. A separate dataset containing six classes (cardboard, paper, plastic, glass, metal, e-waste) was created, excluding non-recyclable categories.



***The dataset was split into train, validation, and test sets, then reorganized into binary and multi-class datasets to support a hierarchical waste classification approach.”***

## **Project Architecture**

#### Input Image
   ↓
#### Binary Classifier
(Recyclable / Non-Recyclable)
   
   ↓
#### If Recyclable
   
   ↓
#### Multi-Class Classifier
(Paper / Plastic / Glass / Metal / Cardboard / E-waste)

**objectives :**

- fristly we will train a CNN model on binary dataset that we created which classifies waste into recyclable/non-recyclable and save the trained model.
- nextly we train another CNN model on multi class dataset to classify waste to type of recyclable waste.and save the model

- now in deployment we will use these two models to support hierarchical waste classification approach.